Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


In [3]:
WITH TOTAL_UNI AS (
	SELECT 
		 INDUS.industry	   AS "industry"	
		,COUNT(*)		   AS "total_number_unicorn"
	FROM public.industries AS INDUS
		LEFT JOIN public.dates AS DATES
			ON INDUS.company_id = DATES.company_id
	WHERE 
		EXTRACT(YEAR FROM DATES.date_joined) IN (2019, 2020,2021)
	GROUP BY 
		INDUS.industry
	ORDER BY 
		"total_number_unicorn" DESC
	LIMIT 3
)

,

YEARLY_UNI AS (
	SELECT 
		 INDUS.industry                        AS "industry"
		,EXTRACT(YEAR FROM DATES.date_joined)  AS "year_joined"
		,COUNT(*)                              AS "num_unicorns"
		,AVG(FUNDS.valuation)                  AS "valuation"
	FROM public.industries AS INDUS
		LEFT JOIN public.dates AS DATES
			ON INDUS.company_id = DATES.company_id
		LEFT JOIN public.funding AS FUNDS
			ON INDUS.company_id = FUNDS.company_id
	WHERE 
		EXTRACT(YEAR FROM DATES.date_joined) IN (2019, 2020, 2021)
	GROUP BY 
		 "industry"
		,"year_joined"
)

SELECT 
	 TOTAL_UNI."industry"		AS "industry"
	,YEARLY_UNI."year_joined"   AS "year"
	,YEARLY_UNI."num_unicorns"	AS "num_unicorns"
	,ROUND(
		YEARLY_UNI."valuation"
		/ 1000000000
		,2
	)                           AS "average_valuation_billions"
FROM TOTAL_UNI
	LEFT JOIN YEARLY_UNI
		ON TOTAL_UNI."industry" = YEARLY_UNI."industry"
ORDER BY 
	 "year" DESC
	,"num_unicorns" DESC

,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58
